![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Semantic Routing

RedisVL provides a `SemanticRouter` interface to utilize Redis' built-in search & aggregation in order to perform
KNN-style classification over a set of `Route` references to determine the best match.

This notebook will go over how to use Redis as a Semantic Router for your applications.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/semantic-router/00_semantic_router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Setup

## Install Packages

In [1]:
%pip install -q "redisvl>=0.4.1" sentence-transformers

Note: you may need to restart the kernel to use updated packages.


## Run a Redis instance

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [17]:
import os
import warnings

warnings.filterwarnings("ignore")

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

# Allow/block list with router

When ChatGPT first launched, there was a famous example where a car dealership accidentally made one of the latest language models available for free to everyone. They assumed users would only ask questions about cars through their chatbot. However, a group of developers quickly realized that the model was powerful enough to answer coding questions, so they started using the dealership's chatbot for free. <br>

To prevent this kind of misuse in your system, adding an allow/block router to the front of your application is essential. Fortunately, this is very easy to implement using `redisvl`. <br>

The code below initializes a vectorizer that will create the vectors that will be stored and initialize the `SemanticRouter` class from `redisvl` that will do the bulk of the configuration required for the router.

In [18]:
from redisvl.extensions.router import Route, SemanticRouter
from redisvl.utils.vectorize import HFTextVectorizer

vectorizer = HFTextVectorizer()

# Semantic router
blocked_references = [
    "things about aliens",
    "corporate questions about agile",
    "anything about the S&P 500",
]

blocked_route = Route(name="block_list", references=blocked_references)

block_router = SemanticRouter(
    name="bouncer",
    vectorizer=vectorizer,
    routes=[blocked_route],
    redis_url=REDIS_URL,
    overwrite=False,
)

# Test it out

In [19]:
user_query = "Why is agile so important?"

route_match = block_router(user_query)

route_match

RouteMatch(name='block_list', distance=0.375402927399)

You can see that we matched to the block list with the example. In an application flow, this value could be checked to short-circuit etc.

This could also be implemented in the positive where examples are provided for topics you'd like to allow.

# Routing with multiple routes

## Define the Routes

Below we define 3 different routes. One for `technology`, one for `sports`, and
another for `entertainment`. Now for this example, the goal here is
surely topic "classification". But you can create routes and references for
almost anything.

Each route has a set of references that cover the "semantic surface area" of the
route. The incoming query from a user needs to be semantically similar to one or
more of the references in order to "match" on the route. Note that each route can have it's own distinct `distance_threshold` that defines what is considered a match for the particular query. 

In [20]:
from redisvl.extensions.router import Route

# Define routes for the semantic router
technology = Route(
    name="technology",
    references=[
        "what are the latest advancements in AI?",
        "tell me about the newest gadgets",
        "what's trending in tech?"
    ],
    metadata={"category": "tech", "priority": 1},
    distance_threshold=0.5
)

sports = Route(
    name="sports",
    references=[
        "who won the game last night?",
        "tell me about the upcoming sports events",
        "what's the latest in the world of sports?",
        "sports",
        "basketball and football"
    ],
    metadata={"category": "sports", "priority": 2},
    distance_threshold=0.7
)

entertainment = Route(
    name="entertainment",
    references=[
        "what are the top movies right now?",
        "who won the best actor award?",
        "what's new in the entertainment industry?"
    ],
    metadata={"category": "entertainment", "priority": 3},
    distance_threshold=0.6
)

## Initialize the SemanticRouter

Like before the ``SemanticRouter`` class will automatically create an index within Redis upon initialization for the route references.

In [21]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Initialize the SemanticRouter
multi_topic_router = SemanticRouter(
    name="topic-router",
    vectorizer=HFTextVectorizer(),
    routes=[technology, sports, entertainment],
    redis_url="redis://localhost:6379",
    overwrite=True # Blow away any other routing index with this name
)

## View the created index

In [22]:
# look at the index specification created for the semantic router
!rvl index info -i topic-router



Index Information:
╭──────────────┬────────────────┬──────────────────┬─────────────────┬────────────╮
│ Index Name   │ Storage Type   │ Prefixes         │ Index Options   │   Indexing │
├──────────────┼────────────────┼──────────────────┼─────────────────┼────────────┤
│ topic-router │ HASH           │ ['topic-router'] │ []              │          0 │
╰──────────────┴────────────────┴──────────────────┴─────────────────┴────────────╯
Index Fields:
╭────────────┬─────────────┬────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬─────────────────┬────────────────╮
│ Name       │ Attribute   │ Type   │ Field Option   │ Option Value   │ Field Option   │ Option Value   │ Field Option   │   Option Value │ Field Option    │ Option Value   │
├────────────┼─────────────┼────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼─────────────────┼────────────────┤
│ route_name │ route

## Simple routing

In [23]:
# Query the router with a statement
route_match = multi_topic_router("Can you tell me about the latest in artificial intelligence?")
route_match

RouteMatch(name='technology', distance=0.419145862261)

In [24]:
# Query the router with a statement and return a miss
route_match = multi_topic_router("are aliens real?")
route_match

RouteMatch(name=None, distance=None)

We can also route a statement to many routes and order them by distance:

In [34]:
# Perform multi-class classification with route_many() -- toggle the max_k and the distance_threshold
route_matches = multi_topic_router.route_many("entertainment and sports", max_k=3)
route_matches

[RouteMatch(name='sports', distance=0.274198234081),
 RouteMatch(name='entertainment', distance=0.521298646927)]

In [35]:
# Toggle the aggregation method -- note the different distances in the result
from redisvl.extensions.router.schema import DistanceAggregationMethod

route_matches = multi_topic_router.route_many("sports and entertainment", aggregation_method=DistanceAggregationMethod.min, max_k=3)
route_matches

[RouteMatch(name='sports', distance=0.263298630714),
 RouteMatch(name='entertainment', distance=0.512374281883)]

Note the different route match distances. This is because we used the `min` aggregation method instead of the default `avg` approach.

## Update the routing config


In [36]:
from redisvl.extensions.router import RoutingConfig

multi_topic_router.update_routing_config(
    RoutingConfig(aggregation_method=DistanceAggregationMethod.min, max_k=3)
)

In [37]:
route_matches = multi_topic_router.route_many("Lebron James")
route_matches

[RouteMatch(name='sports', distance=0.663253903389)]

## Router serialization

In [38]:
multi_topic_router.to_dict()

{'name': 'topic-router',
 'routes': [{'name': 'technology',
   'references': ['what are the latest advancements in AI?',
    'tell me about the newest gadgets',
    "what's trending in tech?"],
   'metadata': {'category': 'tech', 'priority': 1},
   'distance_threshold': 0.5},
  {'name': 'sports',
   'references': ['who won the game last night?',
    'tell me about the upcoming sports events',
    "what's the latest in the world of sports?",
    'sports',
    'basketball and football'],
   'metadata': {'category': 'sports', 'priority': 2},
   'distance_threshold': 0.7},
  {'name': 'entertainment',
   'references': ['what are the top movies right now?',
    'who won the best actor award?',
    "what's new in the entertainment industry?"],
   'metadata': {'category': 'entertainment', 'priority': 3},
   'distance_threshold': 0.6}],
 'vectorizer': {'type': 'hf',
  'model': 'sentence-transformers/all-mpnet-base-v2'},
 'routing_config': {'max_k': 3, 'aggregation_method': 'min'}}

In [42]:
router2 = SemanticRouter.from_dict(multi_topic_router.to_dict(), redis_url="redis://localhost:6379")

11:18:33 redisvl.index.index INFO   Index already exists, not overwriting.


In [43]:
multi_topic_router.to_yaml("router.yaml", overwrite=True)

In [44]:
router3 = SemanticRouter.from_yaml("router.yaml", redis_url="redis://localhost:6379")

11:18:53 redisvl.index.index INFO   Index already exists, not overwriting.


## Clean up the router

In [ ]:
# Use clear to flush all routes from the index
multi_topic_router.clear()
block_router.clear()

In [23]:
# Use delete to clear the index and remove it completely
multi_topic_router.delete()
block_router.delete()

In [25]:
# remove file
!rm -rf router.yaml